# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import os

In [2]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = '''
SELECT 
om.create_time, om.order_number,om.nickname, tmu.true_name, tol.delivery_username, om.user_mobile, tol.delivery_mobile, tmu.id_card_num, tol.delivery_detail_address 
,om.status,om.id as order_id 
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
, tprm.rembursement_status, tos.sort, tos.real_pay_money
,pa.name as activity_name
,top.total_describes
FROM db_digua_business.t_postlease_receivables_monitoring tprm
LEFT JOIN db_digua_business.t_order om on tprm.order_id=om.id
LEFT JOIN db_digua_business.t_order_logistics tol on om.id=tol.order_id
LEFT JOIN db_digua_business.t_member_user tmu on om.user_mobile=tmu.mobile
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE sort=1 and real_pay_money!=0 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静')
-- DATE_FORMAT(om.create_time,'%Y-%m')=DATE_FORMAT(DATE_ADD(CURRENT_DATE,INTERVAL -1 MONTH),'%Y-%m') and 
'''

df_order = query(sql1)
df_order.shape

(53522, 17)

# 合并数据

In [5]:
df = df_order.copy()

# 处理日期

In [6]:
def cle_date(df):
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    df["下单日期"]
    df["月份"]=df["下单日期"].dt.month
    df.shape
    return df


In [7]:
df = cle_date(df)
df['Year1'] = df['create_time'].dt.year
df['Month1'] = df['create_time'].dt.month
df['Year2'] = pd.Timestamp.now().year
df['Month2'] = pd.Timestamp.now().month-1
df1 = df.query('(Year1==Year2) & (Month1==Month2)')
df1

,create_time,order_number,nickname,true_name,delivery_username,user_mobile,delivery_mobile,id_card_num,delivery_detail_address,status,order_id,status2,rembursement_status,sort,real_pay_money,activity_name,total_describes,下单日期,月份,Year1,Month1,Year2,Month2
43982,2025-06-01 00:21:15,A2025060100211445,None,谢志杰,谢志杰,18820584524,18820584524,441827199803233615,汇盈世纪花园8栋1501,4,2782447,租赁中,还款中,1,1.00,芝麻租物-苹果16系列,"[""88890_杨健订单A2025060100211445，msg=未接"", ""88890_...",2025-06-01,6,2025,6,2025,6
43983,2025-06-01 08:39:26,A2025060108392662,None,郝红君,郝红君,15192569050,15192569050,370281199906254263,三里河街道市北京西路明星鞋厂里山海经松川瀚月店,4,2783178,租赁中,还款中,1,1.00,搜索直达-商品推广,"[""修改收货信息：将原收货信息 甜甜 151****9050 山东省 青岛市 胶州市 山东省...",2025-06-01,6,2025,6,2025,6
43984,2025-06-01 09:29:19,A20250601092919101,None,麦勇旺,麦勇旺,13678484770,13678484770,500241200609058011,武陵副食品批发市场2楼名悦汇足浴有限公司,4,2783316,租赁中,还款中,1,1.00,搜索直达-商品推广,"[""修改收货信息：将原收货信息 麦勇旺 136****4770 重庆 重庆郊县 秀山土家族苗...",2025-06-01,6,2025,6,2025,6
43985,2025-06-01 09:47:15,A20250601094715187,None,周倩,周倩,13966881333,13966881333,341223198404081945,亳州市经济技术开发区建投世纪城13栋1303,4,2783401,租赁中,还款中,1,1.00,搜索直达-商品推广,"[""修改收货信息：将原收货信息 周倩 139****1333 安徽省 亳州市 谯城区 安徽省...",2025-06-01,6,2025,6,2025,6
43986,2025-06-01 10:12:44,A2025060110124375,None,杜娅,杜娅,13894561067,13894561067,220581199706261462,桃源新村c2，1楼,4,2783542,租赁中,还款中,1,1.00,端内价格-全新苹果,"[""88888_廖丽敏订单A2025060110124375，msg=铁路上班"", ""888...",2025-06-01,6,2025,6,2025,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48574,2025-06-30 18:23:55,A20250630182354353,None,屈菊芳,屈菊芳,18827258661,18827258661,420527200310130022,大丰区中央公馆1615,4,3074656,租赁中,未到首期还款日,1,1.00,端内价格-全新苹果,"[""系统已发起自动扣款，扣款期数：1+1（注：自动扣款无法保证最终一定成功请注意检查）"", ...",2025-06-30,6,2025,6,2025,6
48575,2025-06-30 21:07:54,A2025063021075329,None,刘建鸿,刘建鸿,18676826340,18676826340,440221199710312232,凯旋城12栋802,4,3076161,租赁中,未到首期还款日,1,468.44,芝麻租物「严选」栏目-好商好品好服务好权益,"[""88888_黄兰娟订单A2025063021075329，msg=用户正忙"", ""888...",2025-06-30,6,2025,6,2025,6
48576,2025-06-30 21:30:47,A20250630213046116,None,陈鑫,陈鑫,18170580634,13507057244,360902200212063020,学府路学林新城10栋1103,4,3076249,租赁中,还款中,1,1.00,芝麻租物-苹果16系列,"[""88888_廖丽敏订单A20250630213046116，msg=都未接"", ""888...",2025-06-30,6,2025,6,2025,6
48782,2025-06-30 17:22:15,A20250630172103350,None,吴曲萌,吴曲萌,15077904208,15077904208,340104198903264521,习友路与集贤路交口恒大华府小区10栋803,4,3073798,租赁中,未到首期还款日,1,1.00,京东活动,"[""88888_周汉鸿订单A20250630172103350，msg=未接"", ""8888...",2025-06-30,6,2025,6,2025,6


# 剔除商家

In [8]:
def tcsj(df):
    dict_status_code = {
        "订单取消": 1,
        "待支付": 2,
        "已退款": 3, 
        "待审核": 4,
        "待发货": 5,
        "待收货": 6,
        "租赁中": 7,
        "已完成": 8
    }
    df["状态编码"]=df["status2"].map(dict_status_code)


    df.sort_values(by=["下单日期","状态编码"],inplace=True)


    #删除重复单号
    df.drop_duplicates(subset=["order_id"],inplace=True)
    #删除身份证空值行
    df.dropna(subset=["id_card_num"],axis=0,inplace=True)
    #去刷单订单
    df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
    df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
    df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
    #删除身份证空值行
    df.dropna(subset=["status2"],axis=0,inplace=True)


    #删除重复订单
    df.drop_duplicates(subset=["order_id"],inplace=True)
    df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


    df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

    return df

# 全部订单

In [9]:
df1 = tcsj(df1)
df1 = df1[['create_time', '下单日期', 'order_number', 'true_name', 'delivery_username', 'user_mobile', 'delivery_mobile', 'id_card_num', 'delivery_detail_address', 'real_pay_money']]
df1.rename(columns={'create_time': '下单时间', 'id_card_num': '身份证号', 'order_number': '订单号', 'true_name': '真实名称', 'delivery_username': '收货名称'
                    , 'user_mobile': '下单手机号', 'delivery_mobile': '收货手机号', 'delivery_detail_address': '详细收货地址', 'real_pay_money': '实付款'}, inplace=True)
df1


,下单时间,下单日期,订单号,真实名称,收货名称,下单手机号,收货手机号,身份证号,详细收货地址,实付款
43982,2025-06-01 00:21:15,2025-06-01,A2025060100211445,谢志杰,谢志杰,18820584524,18820584524,441827199803233615,汇盈世纪花园8栋1501,1.00
43983,2025-06-01 08:39:26,2025-06-01,A2025060108392662,郝红君,郝红君,15192569050,15192569050,370281199906254263,三里河街道市北京西路明星鞋厂里山海经松川瀚月店,1.00
43984,2025-06-01 09:29:19,2025-06-01,A20250601092919101,麦勇旺,麦勇旺,13678484770,13678484770,500241200609058011,武陵副食品批发市场2楼名悦汇足浴有限公司,1.00
43985,2025-06-01 09:47:15,2025-06-01,A20250601094715187,周倩,周倩,13966881333,13966881333,341223198404081945,亳州市经济技术开发区建投世纪城13栋1303,1.00
43986,2025-06-01 10:12:44,2025-06-01,A2025060110124375,杜娅,杜娅,13894561067,13894561067,220581199706261462,桃源新村c2，1楼,1.00
...,...,...,...,...,...,...,...,...,...,...
48576,2025-06-30 21:30:47,2025-06-30,A20250630213046116,陈鑫,陈鑫,18170580634,13507057244,360902200212063020,学府路学林新城10栋1103,1.00
48782,2025-06-30 17:22:15,2025-06-30,A20250630172103350,吴曲萌,吴曲萌,15077904208,15077904208,340104198903264521,习友路与集贤路交口恒大华府小区10栋803,1.00
48783,2025-06-30 17:41:04,2025-06-30,A20250630174104633,刘宇,刘宇,17757492613,17757492613,152127199506271520,内蒙古自治区呼伦贝尔市鄂伦春自治旗大杨树第 二中学二中路4巷6号,564.35
48240,2025-06-30 10:22:03,2025-06-30,A20250630102202280,林燕辉,林燕辉,17268040307,17268040307,350681200310202538,九湖镇水仙花苑王少军园艺,1082.94


## 手机号匹配不一致的订单

In [10]:
df1_dif = df1.query('下单手机号!=收货手机号')
df1_dif_count = df1_dif['订单号'].count()
df1_dif_count

np.int64(301)

## 手机号不一致的手机号的下单时间之前的下单手机号

In [11]:
merge_df = pd.merge(df, df1, left_on='create_time', right_on='下单时间', suffixes=('_df', '_df1'))
merge_df.head(1)

,create_time,order_number,nickname,true_name,delivery_username,user_mobile,delivery_mobile,id_card_num,delivery_detail_address,status,order_id,status2,rembursement_status,sort,real_pay_money,activity_name,total_describes,下单日期_df,月份,Year1,Month1,Year2,Month2,下单时间,下单日期_df1,订单号,真实名称,收货名称,下单手机号,收货手机号,身份证号,详细收货地址,实付款
0,2025-06-01 00:21:15,A2025060100211445,None,谢志杰,谢志杰,18820584524,18820584524,441827199803233615,汇盈世纪花园8栋1501,4,2782447,租赁中,还款中,1,1.0,芝麻租物-苹果16系列,"[""88890_杨健订单A2025060100211445，msg=未接"", ""88890_...",2025-06-01,6,2025,6,2025,6,2025-06-01 00:21:15,2025-06-01,A2025060100211445,谢志杰,谢志杰,18820584524,18820584524,441827199803233615,汇盈世纪花园8栋1501,1.0


In [12]:
# df2 = tcsj(df2)
# df2 = df2[['create_time', '下单日期', 'order_number', 'true_name', 'delivery_username', 'user_mobile', 'delivery_mobile', 'id_card_num', 'delivery_detail_address']]
# df2.rename(columns={'create_time': '下单时间', 'id_card_num': '身份证号', 'order_number': '订单号', 'true_name': '真实名称', 'delivery_username': '收货名称'
#                     , 'user_mobile': '下单手机号', 'delivery_mobile': '收货手机号', 'delivery_detail_address': '详细收货地址'}, inplace=True)
# df2

# 获取逾期订单

In [13]:
f_path = 'F:/需求/截止本月到期订单/'
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    excel_names.append(excel_name)
    excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)

['~$本月到期订单截止_2024-09-27_1.xlsx', '本月到期订单截止_2024-09-27.xlsx', '本月到期订单截止_2024-09-27_1.xlsx', '本月到期订单截止_2024-09-28_1.xlsx', '本月到期订单截止_2024-09-29_1.xlsx', '本月到期订单截止_2024-10-06_1.xlsx', '本月到期订单截止_2024-10-12.xlsx', '本月到期订单截止_2024-10-24.xlsx', '本月到期订单截止_2024092914.xlsx', '本月到期订单截止_2024093010.xlsx', '本月到期订单截止_2024100208.xlsx', '本月到期订单截止_2024100308.xlsx', '本月到期订单截止_2024100312.xlsx', '本月到期订单截止_2024100408.xlsx', '本月到期订单截止_2024100412.xlsx', '本月到期订单截止_2024100508.xlsx', '本月到期订单截止_2024100510.xlsx', '本月到期订单截止_2024100512.xlsx', '本月到期订单截止_2024100608.xlsx', '本月到期订单截止_2024100610.xlsx', '本月到期订单截止_2024100611.xlsx', '本月到期订单截止_2024100612.xlsx', '本月到期订单截止_2024100708.xlsx', '本月到期订单截止_2024100709.xlsx', '本月到期订单截止_2024100710.xlsx', '本月到期订单截止_2024100712.xlsx', '本月到期订单截止_2024100810.xlsx', '本月到期订单截止_2024100909.xlsx', '本月到期订单截止_2024100917.xlsx', '本月到期订单截止_2024100918.xlsx', '本月到期订单截止_2024101009.xlsx', '本月到期订单截止_2024101017.xlsx', '本月到期订单截止_2024101019.xlsx', '本月到期订单截止_2024101109.xlsx', '本月到期订单截止_2024101209.xlsx', '本月到期订单

In [14]:
df_yqs = pd.ExcelFile(f_path + excel_names[-1])
df_yqs.sheet_names
df_yq_1 = df_yqs.parse('逾期1+订单')
df_yq_4 = df_yqs.parse('逾期4+订单')
df_yq_7 = df_yqs.parse('逾期7+订单')
df_yq_15 = df_yqs.parse('逾期15+订单')
df_yq_15.head(1)

,channel_name,activity_name,total_describes,rejected,result,order_id,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,逾期天数,sort,order_method,status2,取消原因,电审拒绝原因,无法联系原因,进件,来源渠道,拒绝理由,归属渠道,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,是否进件,下单月份,预计还款月份


# 匹配逾期订单

## 匹配逾期1+

In [15]:
df2_yq_1 = df_yq_1[df_yq_1['订单号'].isin(df1_dif['订单号'].to_list())]
df2_yq_1_count = df2_yq_1['订单号'].count()
df2_yq_1_count

np.int64(0)

## 匹配逾期4+

In [16]:
df2_yq_4 = df_yq_4[df_yq_4['订单号'].isin(df1_dif['订单号'].to_list())]
df2_yq_4_count = df2_yq_4['订单号'].count()
df2_yq_4_count

np.int64(0)

## 匹配逾期7+

In [17]:
df2_yq_7 = df_yq_7[df_yq_7['订单号'].isin(df1_dif['订单号'].to_list())]
df2_yq_7_count = df2_yq_7['订单号'].count()
df2_yq_7_count

np.int64(0)

## 匹配逾期15+

In [18]:
df2_yq_15 = df_yq_15[df_yq_15['订单号'].isin(df1_dif['订单号'].to_list())]
df2_yq_15_count = df2_yq_15['订单号'].count()
df2_yq_15_count

np.int64(0)

## 逾期汇总

In [19]:
result_yq = pd.DataFrame({
    '总逾期': [df1_dif_count],
    '逾期1+': [df2_yq_1_count],
    '逾期4+': [df2_yq_4_count],
    '逾期7+': [df2_yq_7_count],
    '逾期15+': [df2_yq_15_count]
})

result_yq['1+逾期率'] = df2_yq_1_count/df1_dif_count * 100
result_yq['1+逾期率'] = result_yq['1+逾期率'].apply(lambda x: f'{x:.2f}%')

result_yq['4+逾期率'] = df2_yq_4_count/df1_dif_count * 100
result_yq['4+逾期率'] = result_yq['4+逾期率'].apply(lambda x: f'{x:.2f}%')

result_yq['7+逾期率'] = df2_yq_7_count/df1_dif_count * 100
result_yq['7+逾期率'] = result_yq['7+逾期率'].apply(lambda x: f'{x:.2f}%')

result_yq['15+逾期率'] = df2_yq_15_count/df1_dif_count * 100
result_yq['15+逾期率'] = result_yq['15+逾期率'].apply(lambda x: f'{x:.2f}%')

result_yq = result_yq[['总逾期','逾期1+','1+逾期率','逾期4+','4+逾期率','逾期7+','7+逾期率','逾期15+','15+逾期率']]
result_yq

,总逾期,逾期1+,1+逾期率,逾期4+,4+逾期率,逾期7+,7+逾期率,逾期15+,15+逾期率
0,301,0,0.00%,0,0.00%,0,0.00%,0,0.00%
